In [2]:
import pandas as pd
from IPython.display import display
import sqlite3
import pprint
import re

raw_data_path = r"C:\Users\natha\iCloudDrive\Health\nomie_export\n6.2.0-2023-06-28-10.json"

In [3]:
# JSON is broken into 'books' by the first digit of the day (0-9, 10-19, 20-29, 30/31)
df = pd.read_json(raw_data_path)
df["files"].head()
# display(df["files"]["data/books/2023-06-1"])

awards.json                  [{'id': 'nomie-6', 'name': 'Started Nomie 6', ...
boards.json                  [{'id': 'main', 'label': 'Main', 'elements': [...
dashboards.json              [{'label': 'My Stats', 'widgets': [{'token': {...
data/books/2023-01-1         [{'_id': '944c059f8f', 'note': '#pooped', 'sta...
data/books/2023-01-1_last                             2023-01-20T02:25:18.440Z
Name: files, dtype: object

In [4]:
print(type(df))
print(type(df["files"]))
print(type(df["files"]["data/books/2023-06-1"]))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'list'>


In [5]:
date_pattern = r'\d{4}-\d{2}-\d{1}'
print(re.search(date_pattern, "this is text 2023-06-01").group())

2023-06-0


In [6]:
dates = set()
for book in df["files"].keys():
    date = re.search(date_pattern, book)
    if date:
        dates.add(date.group())
#         print(date.group())

sorted_dates = sorted(dates)
pprint.pprint(sorted_dates)

['2023-01-1',
 '2023-01-2',
 '2023-01-3',
 '2023-02-0',
 '2023-02-1',
 '2023-02-2',
 '2023-03-0',
 '2023-03-1',
 '2023-03-2',
 '2023-03-3',
 '2023-04-0',
 '2023-04-1',
 '2023-04-2',
 '2023-04-3',
 '2023-05-0',
 '2023-05-1',
 '2023-05-2',
 '2023-05-3',
 '2023-06-0',
 '2023-06-1',
 '2023-06-2']


In [7]:
# Setup an empty master dataframe
all_data = pd.DataFrame()

# Read json - every 10 days is a 'book'
for date_range in sorted_dates:
#     print("data/books/" + date_range)
    ten_days = pd.json_normalize(df["files"]["data/books/" + date_range])

    
    ten_days['date'] = pd.to_datetime(ten_days['start']).dt.strftime('%Y-%m-%d')
    ten_days['val'] = ten_days['note'].str.extract('(\d+)')
    ten_days['note'] = ten_days['note'].str.extract('([a-z]+)')
    #print(ten_days.head())
    
    temp_frame = ten_days.drop(columns=['_id', 'start', 'end', 'score', 'lat', 'lng', 'location', 'offset', 'modified', 'source', 'pinned'])
    
    all_data = all_data.append(temp_frame)

    print()
    print(date_range)
    print(temp_frame)






2023-01-1
         note        date  val
0      pooped  2023-01-10  NaN
1      pooped  2023-01-10  NaN
2       sleep  2023-01-10   67
3    caffeine  2023-01-10  750
4       sleep  2023-01-11   24
5      pooped  2023-01-11  NaN
6    caffeine  2023-01-11  750
7       sleep  2023-01-12   60
8    caffeine  2023-01-12  750
9        soda  2023-01-13  NaN
10     pushup  2023-01-13   15
11      squat  2023-01-13   25
12      sleep  2023-01-13   68
13   caffeine  2023-01-13  500
14    alcohol  2023-01-14  NaN
15      sleep  2023-01-14   37
16   caffeine  2023-01-14  750
17       walk  2023-01-14    2
18    anxiety  2023-01-15  NaN
19     pooped  2023-01-15  NaN
20     pooped  2023-01-15  NaN
21      sleep  2023-01-15   16
22       walk  2023-01-15    2
23    anxiety  2023-01-15  NaN
24    alcohol  2023-01-15  NaN
25     pooped  2023-01-15  NaN
26   caffeine  2023-01-15  750
27       read  2023-01-16  NaN
28      sleep  2023-01-16   50
29       walk  2023-01-16    2
30      sleep  2023-01-17   

In [8]:
print(all_data['val'].dtype)
display(all_data)

object


,note,date,val
0,pooped,2023-01-10,NaN
1,pooped,2023-01-10,NaN
2,sleep,2023-01-10,67
3,caffeine,2023-01-10,750
4,sleep,2023-01-11,24
...,...,...,...
53,pushup,2023-06-28,10
54,walk,2023-06-28,2
55,french,2023-06-28,NaN
56,sleep,2023-06-28,79


In [9]:
healthdb = sqlite3.connect('HealthDB.db')
cur = healthdb.cursor()

cur.execute("DROP TABLE IF EXISTS nomie")

all_data.to_sql(name='nomie', con=healthdb)
print("done")

done
